In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os, sys
from scipy.sparse import csr_matrix, coo_matrix
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k
from metrics import precision_at_k as custom_precision, recall_at_k
from utils import prefilter_items
from recommender import MainRecommender

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

C:\Users\Admin\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
data = pd.read_csv('./data/retail_train.csv')

item_features = pd.read_csv('./data/product.csv')
user_features = pd.read_csv('./data/hh_demographic.csv')

item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [3]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head()

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [4]:
item_features.head()

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [5]:
user_features.head()

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16


In [6]:
user_features['age_desc'].unique()

array(['65+', '45-54', '25-34', '35-44', '19-24', '55-64'], dtype=object)

In [7]:
user_features['marital_status_code'].unique()

array(['A', 'U', 'B'], dtype=object)

In [8]:
user_features['household_size_desc'].unique()

array(['2', '3', '4', '1', '5+'], dtype=object)

# 1.Filter items

In [9]:
data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [10]:
n_items_before = data_train['item_id'].nunique()

data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)

n_items_after = data_train_filtered['item_id'].nunique()
print(f'Decreased # items from {n_items_before} to {n_items_after}')

D:\courses\recommender_system\practice\utils.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 86865 to 5001


# 2. Prepare data set

## Prepare csr train matrix

In [11]:
user_item_matrix, sparse_user_item = MainRecommender.prepare_matrix(data_train_filtered)
user_item_matrix.head()

item_id,117847,818981,819255,819308,819400,819487,819590,819594,819840,819845,...,15926775,15926844,15926886,15972074,15972298,15972565,15972790,16100266,16729299,16729415
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Prepare csr test matrix

In [12]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix, test_sparse_user_item = MainRecommender.prepare_matrix(data_test)
test_user_item_matrix.head()

item_id,32392,34873,42852,43094,44522,45507,49812,53516,58612,62804,...,17284297,17284423,17291184,17291665,17320698,17320911,17321092,17328261,17329749,17330255
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
id_to_itemid, id_to_userid, itemid_to_id, userid_to_id = MainRecommender.prepare_dicts(user_item_matrix)

# 3. Prepare user and item features

In [14]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head()

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
user_feat.shape

(2497, 7)

In [16]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)
item_feat.head()

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
117847,450.0,NUTRITION,National,REFRIGERATED,SOY/RICE MILK,64 OZ
818981,194.0,GROCERY,National,COLD CEREAL,ALL FAMILY CEREAL,10.4 OZ
819255,1007.0,MEAT-PCKGD,National,BREAKFAST SAUSAGE/SANDWICHES,ROLLS - PORK,1 LB
819308,2879.0,MEAT,National,BEEF,CHOICE BEEF,
819400,584.0,GROCERY,National,DOG FOODS,DOG TREATS (SOFT TREATS),5.6 OZ


In [17]:
item_feat.shape

(5001, 6)

# Encoding features

In [18]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [19]:
user_feat_lightfm.head()

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
item_feat_lightfm.head()

,manufacturer_2.0,manufacturer_5.0,manufacturer_16.0,manufacturer_20.0,manufacturer_26.0,manufacturer_33.0,manufacturer_35.0,manufacturer_36.0,manufacturer_42.0,manufacturer_43.0,...,curr_size_of_product_L 16 OZ,curr_size_of_product_L 7.75 OZ,curr_size_of_product_L 13.25 OZ,curr_size_of_product_LB,curr_size_of_product_N 12 OZ,curr_size_of_product_N 40 OZ,curr_size_of_product_PINT,curr_size_of_product_PK,curr_size_of_product_PT,curr_size_of_product_QT
item_id,,,,,,,,,,,,,,,,,,,,,
117847,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
818981,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
819255,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
819308,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
819400,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Init model

In [21]:
def grid_search(no_components=40, loss='bpr', learning_rate=0.01, item_alpha=0.4, user_alpha=0.1, k=5, n=15, max_sampled=100):
    """Model best parapmeters search."""
    
    model = LightFM(no_components=no_components,
                    loss=loss,
                    learning_rate=learning_rate,
                    item_alpha=item_alpha,
                    user_alpha=user_alpha,
                    random_state=42,
                    k=k,
                    n=n,
                    max_sampled=max_sampled)
    
    model.fit((sparse_user_item > 0) * 1,
              sample_weight=coo_matrix(user_item_matrix),
              user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
              item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
              epochs=20,
              num_threads=20,
              verbose=True)
    
    train_precision = precision_at_k(model, sparse_user_item,
                                     user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                     item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                     k=k).mean()
    
    print(f'Result for model with parameters: \nno_components={no_components}, \nloss={loss},\
          \nlearning_rate={learning_rate}, \nitem_alpha={item_alpha}, \nuser_alpha={user_alpha}')
    print(f'Train precision {train_precision}')
    
    users_ids_row = data_train_filtered['user_id'].apply(lambda x: userid_to_id[x]).values.astype(int)
    items_ids_row = data_train_filtered['item_id'].apply(lambda x: itemid_to_id[x]).values.astype(int)
    
    predictions = model.predict(user_ids=users_ids_row,
                                item_ids=items_ids_row,
                                user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                num_threads=10)
    
    data_train_filtered['score'] = predictions
    
    predict_result = data_train_filtered[['user_id', 'item_id', 'score']][data_train_filtered.item_id != 999999]. \
        drop_duplicates().sort_values(by=['user_id', 'score'], ascending=False).groupby('user_id')['item_id']. \
        unique().reset_index()
    
    df_result_for_metrics = result.merge(predict_result, on='user_id', how='inner')
    
    precision = df_result_for_metrics.apply(lambda row: custom_precision(row['item_id'], row['actual'], k=5), axis=1).\
        mean()
    
    print(f'Precision: {precision}')

Result for model with parameters: no_components=40, loss=bpr,          learning_rate=0.1, item_alpha=0.04, user_alpha=0.01

In [23]:
no_components = [40]
losses = ['bpr']
learning_rates = [0.1]
item_alphas = [0.04]
user_alphas = [0.01]

for components_number in no_components:
    for loss in losses:
        for learning_rate in learning_rates:
            for item_alpha in item_alphas:
                for user_alpha in user_alphas:
                    grid_search(no_components=components_number,
                                loss=loss,
                                learning_rate=learning_rate,
                                item_alpha=item_alpha,
                                user_alpha=user_alpha)

Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 20/20 [02:34<00:00,  7.74s/it]


Result for model with parameters: 
no_components=40, 
loss=bpr,          
learning_rate=0.1, 
item_alpha=0.04, 
user_alpha=0.01
Train precision 0.4385262429714203
Precision: 0.15047619047618843


In [24]:
def fit_model(no_components=40, loss='bpr', learning_rate=0.1, item_alpha=0.04, user_alpha=0.01, k=5, n=15, max_sampled=100):
    """Model best parapmeters search."""
    
    model = LightFM(no_components=no_components,
                    loss=loss,
                    learning_rate=learning_rate,
                    item_alpha=item_alpha,
                    user_alpha=user_alpha,
                    random_state=42,
                    k=k,
                    n=n,
                    max_sampled=max_sampled)
    
    model.fit((sparse_user_item > 0) * 1,
#               sample_weight=coo_matrix(user_item_matrix),
              user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
              item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
              epochs=20,
              num_threads=20,
              verbose=True)
    
    train_precision = precision_at_k(model, sparse_user_item,
                                     user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                     item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                     k=k).mean()
    
    print(f'Result for model with parameters: \nno_components={no_components}, \nloss={loss},\
          \nlearning_rate={learning_rate}, \nitem_alpha={item_alpha}, \nuser_alpha={user_alpha}')
    print(f'Train precision {train_precision}')
    
    users_ids_row = data_train_filtered['user_id'].apply(lambda x: userid_to_id[x]).values.astype(int)
    items_ids_row = data_train_filtered['item_id'].apply(lambda x: itemid_to_id[x]).values.astype(int)
    
    predictions = model.predict(user_ids=users_ids_row,
                                item_ids=items_ids_row,
                                user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                num_threads=10)
    
    data_train_filtered['score'] = predictions
    
    predict_result = data_train_filtered[['user_id', 'item_id', 'score']][data_train_filtered.item_id != 999999]. \
        drop_duplicates().sort_values(by=['user_id', 'score'], ascending=False).groupby('user_id')['item_id']. \
        unique().reset_index()
    
    df_result_for_metrics = result.merge(predict_result, on='user_id', how='inner')
    
    precision = df_result_for_metrics.apply(lambda row: custom_precision(row['item_id'], row['actual'], k=5), axis=1).\
        mean()
    
    print(f'Precision: {precision}')

In [25]:
fit_model()

Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 20/20 [02:28<00:00,  7.42s/it]


Result for model with parameters: 
no_components=40, 
loss=bpr,          
learning_rate=0.1, 
item_alpha=0.04, 
user_alpha=0.01
Train precision 0.17420905828475952
Precision: 0.06515599343185591
